# DB2-Salesforce connector: Citation information updates

In [1]:
# API settings
api_url = '/services/data/v43.0/sobjects'
external_id = 'Record_ID__c'
object_id = 'nanoHUB_citations__c'

In [2]:
import sys
import os
sys.path.append('/home/users/wang2506/nanohub_salesforce_integ/salesforce')
    
import pandas as pd
import datetime

## Obtain tool information from DB2 

In [3]:
# Basic tool information
sql_query = "select *, c.id as c_id from jos_citations as c \
            left join jos_citations_types as t \
            on c.type = t.id \
            where uid=2143"

# display
print(sql_query)

select *, c.id as c_id from jos_citations as c             left join jos_citations_types as t             on c.type = t.id             where uid=2143


In [4]:
from nanoHUB.application import Application

application = Application.get_instance()
nanohub_db = application.new_db_engine('nanohub')

salesforce = application.new_salesforce_engine()
db_s = salesforce

df = pd.read_sql_query(sql_query, nanohub_db)

nanoHUB - Serving Students, Researchers & Instructors


2021-08-25 23:48:35,517| INFO    | 0 keys loaded from agent


2021-08-25 23:48:35,517 - [INFO] sshtunnel.SSHTunnelForwarder [sshtunnel.get_agent_keys:1060]: 0 keys loaded from agent


2021-08-25 23:48:35,521| INFO    | 0 key(s) loaded


2021-08-25 23:48:35,521 - [INFO] sshtunnel.SSHTunnelForwarder [sshtunnel.get_keys:1117]: 0 key(s) loaded


...

2021-08-25 23:48:35,524 - [INFO] sshtunnel.SSHTunnelForwarder [sshtunnel.__init__:978]: Connecting to gateway: db2.nanohub.org:22 as user 'saxenap'


2021-08-25 23:48:35,527 - [INFO] nanoHUB.containers.dataaccess [connection.get_connection_for:101]: Started SSH Tunnel with db2.nanohub.org


2021-08-25 23:48:35,837| INFO    | Opening tunnel: 0.0.0.0:51819 <> 127.0.0.1:3306


2021-08-25 23:48:35,837 - [INFO] sshtunnel.SSHTunnelForwarder [sshtunnel._serve_forever_wrapper:1433]: Opening tunnel: 0.0.0.0:51819 <> 127.0.0.1:3306


Obtained Salesforce access token ...... True


In [5]:
# convert and clean some columns, so we can, for example, sort values
df['date_publish'] = pd.to_datetime(df['date_publish'], errors='coerce') 
df['created'] = pd.to_datetime(df['created'], errors='coerce') 
df['date_submit'] = pd.to_datetime(df['date_submit'], errors='coerce') 
df['date_accept'] = pd.to_datetime(df['date_accept'], errors='coerce') 

In [6]:
# display
display(df.head(1))
display('size='+str(df.shape))

,id,uid,affiliated,fundedby,created,address,author,booktitle,chapter,cite,...,custom2,custom3,custom4,pdf_url,id,type,type_title,type_desc,type_export,c_id
0,2240,2143,1,-1,NaT,,Piyush Dak{{50986}};Aida Ebrahimi{{57810}};Vik...,,,,...,None,None,None,https://nanohub.org/nanoHUBCitations/citmanage...,2,journal,Journal,None,None,2240


'size=(2517, 68)'

## Match data with Salesforce format

In [7]:
df_sf = pd.DataFrame()

# Make sure NaN and NaT values are taken care of here

df_sf['Record_ID__c']      = df['c_id']
df_sf['URL__c']            = df['url']
df_sf['NCN_Affiliated__c'] = df['affiliated'].apply(lambda x: 1 if x==1 else 0)
df_sf['Author_List__c']    = df['author']
df_sf['Journal__c']        = df['journal']
df_sf['Year__c']           = df['year']
df_sf['Type__c']           = df['ref_type']
df_sf['Notes__c']          = df['notes']
df_sf['Name']              = df['title'].apply(lambda x: x[:70])
df_sf['Full_title__c']     = df['title']
df_sf['Format__c']         = df['type_title']

sf_original_fields = df_sf.columns

# display
df_sf.head(2).T

,0,1
Record_ID__c,2240,1474
URL__c,http://scholar.google.com/scholar?hl=en&q=Drop...,http://scholar.google.com/scholar?hl=en&q=http...
NCN_Affiliated__c,1,0
Author_List__c,Piyush Dak{{50986}};Aida Ebrahimi{{57810}};Vik...,V.C. Stodden
Journal__c,Biosensors,Columbia University Academic Commons
Year__c,2016,2013
Type__c,R,E
Notes__c,,Conference: Open Repositories 2013; \r\nPrince...
Name,"Droplet-based Biosensing For Lab-on-a-Chip, Op...",Re-use And Reproducibility: Opportunities And ...
Full_title__c,"Droplet-based Biosensing For Lab-on-a-Chip, Op...",Re-use And Reproducibility: Opportunities And ...


## To Salesforce Sales Cloud CRM

In [8]:
# specify Salesforce object ID and external ID
db_s.object_id = object_id
db_s.external_id = external_id

In [9]:
# send data to Salesforce
db_s.send_data(df_sf)

[Success] Bulk job creation successful. Job ID = 7505w00000ZJKRgAAP
hello


[Success] CSV upload successful. Job ID = 7505w00000ZJKRgAAP


[Success] Closing job successful. Job ID = 7505w00000ZJKRgAAP


In [10]:
# check status
db_s.check_bulk_status()

{'id': '7505w00000ZJKRgAAP',
 'operation': 'upsert',
 'object': 'nanoHUB_citations__c',
 'createdById': '0055w00000DM5bOAAT',
 'createdDate': '2021-08-26T03:48:38.000+0000',
 'systemModstamp': '2021-08-26T03:48:41.000+0000',
 'state': 'UploadComplete',
 'externalIdFieldName': 'Record_ID__c',
 'concurrencyMode': 'Parallel',
 'contentType': 'CSV',
 'apiVersion': 47.0,
 'jobType': 'V2Ingest',
 'lineEnding': 'LF',
 'columnDelimiter': 'COMMA',
 'numberRecordsProcessed': 0,
 'numberRecordsFailed': 0,
 'retries': 0,
 'totalProcessingTime': 0,
 'apiActiveProcessingTime': 0,
 'apexProcessingTime': 0}

In [11]:
from pprint import pprint
pprint(db_s.check_bulk_failed_results())

''
